<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_2/ch06/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 6 RNN with gate added

**6.1 Problem of RNN**

*6.1.3 Cause of gradient loss and gradient explosion*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 2 # mini-batch size
H = 3 # Dimension of hidden state vector
T = 20 # length of Time Series Data

dh = np.ones((N, H))
np.random.seed(3) # seed fix
Wh = np.random.randn(H, H)

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T)
    norm = np.sqrt(np.sum(dh ** 2)) / N
    norm_list.append(norm)

plt.plot(np.arange(len(norm_list)), norm_list)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 2 # mini-batch size
H = 3 # Dimension of hidden state vector
T = 20 # length of Time Series Data

dh = np.ones((N, H))
np.random.seed(3) # seed fix
Wh = np.random.randn(H, H) * 0.5

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T)
    norm = np.sqrt(np.sum(dh ** 2)) / N
    norm_list.append(norm)

plt.plot(np.arange(len(norm_list)), norm_list)
plt.show()